In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.SN_Curve import SN_Curve_fatpack, SN_Curve_qats
from utils.extract_and_preprocess_data import extract_and_preprocess_data
from utils.setup_custom_logger import setup_custom_logger
from utils.read_simulation_file import read_bladed_file
from utils.save_damage_table import save_damage_table
from utils.create_geo_matrix import create_geo_matrix
from utils.get_stress_range_and_count import get_stress_range_and_count_qats_pckg, get_stress_range_and_count_rainflow_pckg
import numpy as np
import sys
import pandas as pd

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [30 / 2.54, 20 / 2.54]


In [3]:
# Get relevant data_paths for DLC and simulation result files
DLC_IDs = ['DLC12'] #, 'DLC64a', 'DLC64b', 'DLC24a', 'DLC31', 'DLC41a', 'DLC41b']
data_path              = r'C:\Appl\TDI\getting_started\fatpack_test\data'
DLC_file               = data_path +  r'\Doc-0081164-HAL-X-13MW-DGB-A-OWF-Detailed DLC List-Fatigue Support Structure Load Assessment_Rev7.0.xlsx'
sim_res_cluster_folder = data_path +  r'\Doc-0089427-HAL-X-13MW DB-A OWF-ILA3_JLO-model_fatigue_timeseries_all_elevations'
geometry_file          = data_path +  r'\DA_P53_CD.xlsx'

geometry      = pd.read_excel(geometry_file).drop([0,2,3,4], axis=0) # TODO drop can be used for testing only a set of geometries
n_geometries  = geometry.shape[0]
point_angles  = [float(i) for i in range(0,359,15)] 
# TODO transform between "global turbine frame" and the compass frame, as the SCF angles are given in the latter
cluster_ID    = 'JLO'
out_file_type = 'mat' # or npy 
logger        = setup_custom_logger('Main') # logger.info('Message') and logger.error('Message')


In [4]:
DLC_ID = DLC_IDs[0]

In [5]:
range_finder_func  = get_stress_range_and_count_qats_pckg
curve              = SN_Curve_qats('D')
geo_matrix         = create_geo_matrix(geometry, point_angles, curve) # better matrix to pass to the main function

In [6]:
df, probs, n_cases, n_timesteps = extract_and_preprocess_data(DLC_file, DLC_ID, cluster_ID, sim_res_cluster_folder)

In [7]:
arguments = [(df.results_files[i], 
              df.descr_files[i], 
              point_angles, 
              geo_matrix, 
              curve,
              range_finder_func
              ) for i in range(n_cases)]

In [16]:
from scipy import stats
def calculate_DEM_case_i(binary_file_i, description_file_i, point_angles, geo_matrix, curve, range_finder_func):
    '''
    Calculates in-place Damage Equivalent (bending) Moment of a time series 
    - Reads simulation files from vendor and extracts moments, forces time series
    - Transforms time series into resultant moment for the relevant sectors we are looking at
    - Uses rainflow counting to find the various moment ranges
    - Calculates DEM
    ''' 
    content_reshaped = read_bladed_file(binary_file_i, description_file_i) # (n,m) numpy array with n = no. of quantities of data,  m = no. of timesteps for each data quantity 
    DEM_sum = np.zeros( (len(geo_matrix), len(point_angles)) )
    
    for geo_idx, geo_dict in geo_matrix.items():
        pos = (geo_dict['mx_col'] - 1, geo_dict['my_col'] - 1, geo_dict['fz_col'] - 1) # columns in DLC file to collect moments and forces, using -1 to fit Python indexing
        
        moments_x_timeseries = content_reshaped[[pos[0]], :] # Moments as (1, timesteps) array - hence the [[],:] type slice
        moments_y_timeseries = content_reshaped[[pos[1]], :] # Moments as (1, timesteps) array
        forces_z_case_i      = content_reshaped[[pos[2]], :] # Axial F as (1, timesteps) array 

        # Resultant moment for the current case -> (theta, timestep)-shaped array    
        actual_angles_rad = np.deg2rad(geo_dict['actual_angles'])
        
        res_moments_timeseries_case_i = np.sin([actual_angles_rad]).T.dot(moments_x_timeseries) - np.cos([actual_angles_rad]).T.dot(moments_y_timeseries)
        res_force_timeseries_case_i = np.repeat(forces_z_case_i, len(actual_angles_rad), axis = 0)
                    
        for ang_idx, timeseries_case_i_ang_j in enumerate(res_moments_timeseries_case_i):
            
            # ranges comes as (n_ranges, 2) sized matrix with col 0 = ranges and col 1 = counts
            ranges = range_finder_func(timeseries_case_i_ang_j, k = 128)   
            DEM_sum[[geo_idx], [ang_idx]] = ((ranges[:,[0]].T)**5.0).dot(ranges[:,[1]]) * 6.0 # PER HOUR - calculating SUM_i^k (n_i * dM_i^m) by manipulating the range matrix and using dot product for summation 
            
            # # TODO debug
            # print(ranges.shape)  
            # print(DEM_sum[[geo_idx], [ang_idx]])            
            # sys.exit()
    
    return DEM_sum # (n_geo, n_angles) shaped array

In [17]:
DEM_sum_table = np.zeros((n_cases, n_geometries, len(point_angles))) # pre-allocate output matrix of the current DLC
if False:
    from multiprocessing import Pool
    with Pool() as p: 
        DEM_sums = p.starmap(calculate_DEM_case_i, arguments) # returns a list of damages of len n_cases 
      
    DEM_sum_table = np.array(DEM_sums)  
else:
    for case_i in range(n_cases):
        DEM_sum_table[[case_i], :, :] = calculate_DEM_case_i(*arguments[case_i])

In [18]:
combined_DEM_DLC_i = np.zeros((n_geometries, len(point_angles)))
weights = np.array([probs])
for ang_idx in range(len(point_angles)):
    combined_DEM_DLC_i[:, [ang_idx]] = np.dot( weights, DEM_sum_table[:,:, ang_idx]).T

In [30]:
combined_DEM_DLC_i
test_df = pd.DataFrame(data = combined_DEM_DLC_i, columns = [f'DEM {i} deg' for i in point_angles])
test_df

,DEM 0.0 deg,DEM 15.0 deg,DEM 30.0 deg,DEM 45.0 deg,DEM 60.0 deg,DEM 75.0 deg,DEM 90.0 deg,DEM 105.0 deg,DEM 120.0 deg,DEM 135.0 deg,...,DEM 210.0 deg,DEM 225.0 deg,DEM 240.0 deg,DEM 255.0 deg,DEM 270.0 deg,DEM 285.0 deg,DEM 300.0 deg,DEM 315.0 deg,DEM 330.0 deg,DEM 345.0 deg
0,1.046117e+44,1.056250e+44,9.996766e+43,9.004959e+43,8.009943e+43,7.319754e+43,7.035115e+43,7.129573e+43,7.507004e+43,8.118825e+43,...,9.996766e+43,9.004959e+43,8.009943e+43,7.319754e+43,7.035115e+43,7.129573e+43,7.507004e+43,8.118825e+43,8.944782e+43,9.816648e+43


In [20]:
T_lifetime = 25.0
N_equivalent = 10.0**7
wohler_exp = 5.0

In [31]:
DEM_tot = (T_lifetime / N_equivalent * combined_DEM_DLC_i.max() )**(1/wohler_exp)

In [32]:
print(f'{DEM_tot / 10**6} MNm')

48.34385926603874 MNm


In [ ]:
# TODO next steps is to test interpolation